In [39]:
pwd

'C:\\Users\\Ramya\\Documents'

In [42]:
cd "C:\Users\Ramya\Desktop\Ramya\GenomicDataScience\Algorithms for GDS"

C:\Users\Ramya\Desktop\Ramya\GenomicDataScience\Algorithms for GDS


In [43]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [44]:
t = readGenome("chr1.GRCh38.excerpt.fasta") # reading the excerpt of human chromosome 1 for further analysis

Adapt the editDistance function we saw in practical (copied below) to answer questions 1 and 2 below. 
Your function should take arguments p (pattern), t (text) and should return the edit distance of the match 
between P and T with the fewest edits.

Hint: In the "A new solution to approximate matching" video we saw that the best approximate match of 
    P =GCGTATGCwithin T =TATTGGCTATACGGTThad 2 edits. 
    You can use this and other small examples to double-check that your function is working.

In [47]:
def editDistance_approx_match(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = 0
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    
    # Edit distance is the value in the bottom right corner of the matrix
    return min(D[-1])

In [48]:
# Example 1
P = 'GCGTATGC'
T = 'TATTGGCTATACGGTT'
ed = editDistance_approx_match(P, T)
print(ed)

2


Question 1

What is the edit distance of the best match between pattern GCTGATCGATCGTACG and the excerpt of human chromosome 1? 
(Don't consider reverse complements.)

In [49]:
P = 'GCTGATCGATCGTACG'
T = t
ed = editDistance_approx_match(P, T)
print(ed)

3


Question 2

What is the edit distance of the best match between pattern GATTTACCAGATTGAG and the excerpt of human chromosome 1? 
(Don't consider reverse complements.)

In [50]:
P = 'GATTTACCAGATTGAG'
T = t
ed = editDistance_approx_match(P, T)
print(ed)

2


Question 3

In a practical, we saw a function for finding the longest exact overlap (suffix/prefix match) between two strings. 
The function is copied below.
Say we are concerned only with overlaps that (a) are exact matches (no differences allowed), and (b) are at least k bases long. 
To make an overlap graph, we could call overlap(a, b, min_length=k) on every possible pair of reads from the dataset. 
Unfortunately, that will be very slow!

Consider this: Say we are using k=6, and we have a read a whose length-6 suffix is GTCCTA. 
Say GTCCTA does not occur in any other read in the dataset. 
In other words, the 6-mer GTCCTA occurs at the end of read a and nowhere else. 
It follows that a's suffix cannot possibly overlap the prefix of any other read by 6 or more characters.

Put another way, if we want to find the overlaps involving a suffix of read a and a prefix of some other read, 
we can ignore any reads that don't contain the length-k suffix of a. This is good news because it can save us a lot of work!

Here is a suggestion for how to implement this idea. You don't have to do it this way, but this might help you. 
Let every k-mer in the dataset have an associated Python set object, which starts out empty. 
We use a Python dictionary to associate each k-mer with its corresponding set. 
(1) For every k-mer in a read, we add the read to the set object corresponding to that k-mer. 
If our read is GATTA and k=3, we would add GATTA to the set objects for GAT, ATT and TTA. 
We do this for every read so that, at the end, each set contains all reads containing the corresponding k-mer. 
(2) Now, for each read a, we find all overlaps involving a suffix of a. 
To do this, we take a's length-k suffix, find all reads containing that k-mer (obtained from the corresponding set) 
and call overlap(a, b, min_length=k) for each.

The most important point is that we do not call overlap(a, b, min_length=k) if b does not contain the length-k suffix of a.

Download and parse the read sequences from the provided Phi-X FASTQ file. 
We'll just use their base sequences, so you can ignore read names and base qualities. 
Also, no two reads in the FASTQ have the same sequence of bases. This makes things simpler.

https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq

Next, find all pairs of reads with an exact suffix/prefix match of length at least 30. 
Don't overlap a read with itself; if a read has a suffix/prefix match to itself, ignore that match. 
Ignore reverse complements.

Hint 1: Your function should not take much more than 15 seconds to run on this 10,000-read dataset, 
and maybe much less than that. (Our solution takes about 3 seconds.) 
If your function is much slower, there is a problem somewhere.

Hint 2: Remember not to overlap a read with itself. If you do, your answers will be too high.
    
Hint 3: You can test your implementation by making up small examples, then checking that 
(a) your implementation runs quickly, and (b) you get the same answer as if you had simply called overlap(a, b, min_length=k) 
on every pair of reads. We also have provided a couple examples you can check against.
Picture the overlap graph corresponding to the overlaps just calculated. How many edges are in the graph? 
In other words, how many distinct pairs of reads overlap?

In [52]:
#Parse the file and get the reads out in a list
def parse_repeats(filename):
    repeats = []
    from Bio import SeqIO
    with open(filename, "rU") as handle:
        for record in SeqIO.parse(handle, "fastq"):
            repeats.append(str(record.seq))
    return repeats

In [53]:
seqs = parse_repeats("ERR266411_1.for_asm.fastq")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: DeprecationWarning: 'U' mode is deprecated


In [145]:
def overlap(a, b, min_length):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [177]:
def overlap_all_pairs(reads, k):
    from collections import defaultdict
    kmer_set = [] # get the kmers into a list of tuples with the kmer and the reads from they came
    kmer_dict = defaultdict(set) # convert the kmer_set to a dictionary with unique kmers and set of reads as values 
    match_pairs = [] # list to get the unique suffix-prefix pairs
    # create the kmer_set
    for read in reads:
        for i in range(len(read)-k+1):
            kmer = read[i:i+k]
            kmer_set.append((kmer, read))
    # create the kmer_dict as a defaultdict set    
    for key, value in kmer_set:
        kmer_dict[key].add(value)
    # for each read, get the suffix, look in the dict, get the corresponding set and loop through the set 
    # with the overlap suffix-prefix function
    for read in reads:
        a = read[-k:]
        if a in kmer_dict:
            chk_reads = kmer_dict[a]
            for cr in chk_reads:
                if read != cr: # make sure the suffix, prefix are not from the same read
                    olap = overlap(read, cr, k)
                    if olap != 0:
                        match_pairs.append((read, cr))
    return len(match_pairs)

In [176]:
# Example 1
reads = ['ABCDEFG', 'EFGHIJ', 'HIJABC']
overlap_all_pairs(reads, 3)



[('ABCDEFG', 'EFGHIJ'), ('EFGHIJ', 'HIJABC'), ('HIJABC', 'ABCDEFG')]

In [168]:
overlap_all_pairs(reads, 4)

[]

In [170]:
# Example 2
re = ['CGTACG', 'TACGTA', 'GTACGT', 'ACGTAC', 'GTACGA', 'TACGAT']
overlap_all_pairs(re, 4)

[('CGTACG', 'TACGAT'),
 ('CGTACG', 'TACGTA'),
 ('CGTACG', 'GTACGT'),
 ('CGTACG', 'GTACGA'),
 ('TACGTA', 'CGTACG'),
 ('TACGTA', 'ACGTAC'),
 ('GTACGT', 'TACGTA'),
 ('GTACGT', 'ACGTAC'),
 ('ACGTAC', 'GTACGA'),
 ('ACGTAC', 'GTACGT'),
 ('ACGTAC', 'CGTACG'),
 ('GTACGA', 'TACGAT')]

In [171]:
overlap_all_pairs(re, 5)

[('CGTACG', 'GTACGT'),
 ('CGTACG', 'GTACGA'),
 ('TACGTA', 'ACGTAC'),
 ('GTACGT', 'TACGTA'),
 ('ACGTAC', 'CGTACG'),
 ('GTACGA', 'TACGAT')]

In [178]:
overlap_all_pairs(seqs, 30)

904746

Question 4

Picture the overlap graph corresponding to the overlaps computed for the previous question. How many nodes in this graph have at least one outgoing edge? 
(In other words, how many reads have a suffix involved in an overlap?)

In [182]:
def overlap_all_suffixread(reads, k):
    from collections import defaultdict
    kmer_set = []
    kmer_dict = defaultdict(set)
    match_pairs = []
    read_suffix = [] # list of reads whose suffix has an overlap
    for read in reads:
        for i in range(len(read)-k+1):
            kmer = read[i:i+k]
            kmer_set.append((kmer, read))
    for key, value in kmer_set:
        kmer_dict[key].add(value)
    for read in reads:
        a = read[-k:]
        if a in kmer_dict:
            chk_reads = kmer_dict[a]
            for cr in chk_reads:
                if read != cr:
                    olap = overlap(read, cr, k)
                    if olap != 0:
                        match_pairs.append((read, cr))
                        if read not in read_suffix:
                            read_suffix.append(read)
    return len(read_suffix)

In [183]:
overlap_all_suffixread(seqs, 30)

7161